In [ ]:
!pip install dython  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.2 MB 5.1 MB/s 
     |████████████████████████████████| 38.1 MB 1.4 MB/s 
     |████████████████████████████████| 930 kB 57.0 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

from time import time
from datetime import timedelta

import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.preprocessing import minmax_scale

from sklearn.svm import LinearSVR, SVR, SVC

from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, KFold, RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix,\
        accuracy_score, classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,GradientBoostingClassifier,StackingClassifier,ExtraTreesClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import PowerTransformer

from matplotlib import pyplot as plt 
import seaborn as sns
%matplotlib inline
import statsmodels.api as sm


from IPython.core.interactiveshell import InteractiveShell
pd.set_option('precision', 3)
InteractiveShell.ast_node_interactivity = "all"
sns.set()

Setting a seed for the random number generators:

In [ ]:
np.random.seed(1)

In [ ]:
flights = pd.read_csv('airlines2.csv')
flights.head()

,Unnamed: 0,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Length,Time,Delay,Speed
0,1,OO,4492,GJT,SLC,6,0.107,-1.501,0,1.263
1,2,EV,5364,MGM,MEM,6,-1.884,-0.874,0,-0.582
2,3,FL,1016,BOS,MCO,7,-0.191,1.160,0,-0.887
3,4,F9,231,BNA,DEN,5,0.415,0.812,0,-0.551
4,5,AS,662,SEA,DFW,5,-0.089,1.396,0,-0.939


We do get_dummies for the categorical variables:

In [ ]:
  # Transform categorical variables into numbers because we will be working with decision trees
  X = flights.drop(['Length', 'Time'], axis=1) # numerical variables
  data = flights.drop(['Airline', 'Flight', 'AirportFrom','AirportTo', 'DayOfWeek','Delay'], axis=1) # categorical variables
  X = pd.get_dummies(X)
  data = pd.concat([data, X], axis=1)
  data

,Unnamed: 0,Length,Time,Speed,Unnamed: 0,Flight,DayOfWeek,Delay,Speed,Airline_9E,...,AirportTo_TXK,AirportTo_TYR,AirportTo_TYS,AirportTo_UTM,AirportTo_VLD,AirportTo_VPS,AirportTo_WRG,AirportTo_XNA,AirportTo_YAK,AirportTo_YUM
0,1,0.107,-1.501,1.263,1,4492,6,0,1.263,0,...,0,0,0,0,0,0,0,0,0,0
1,2,-1.884,-0.874,-0.582,2,5364,6,0,-0.582,0,...,0,0,0,0,0,0,0,0,0,0
2,3,-0.191,1.160,-0.887,3,1016,7,0,-0.887,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0.415,0.812,-0.551,4,231,5,0,-0.551,0,...,0,0,0,0,0,0,0,0,0,0
4,5,-0.089,1.396,-0.939,5,662,5,0,-0.939,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26899,26900,-0.108,0.146,-0.406,26900,374,4,0,-0.406,0,...,0,0,0,0,0,0,0,0,0,0
26900,26901,0.663,-0.769,0.866,26901,1294,4,1,0.866,0,...,0,0,0,0,0,0,0,0,0,0
26901,26902,0.148,0.752,-0.620,26902,1400,7,0,-0.620,0,...,0,0,0,0,0,0,0,0,0,0
26902,26903,1.100,-0.332,0.744,26903,1138,7,1,0.744,0,...,0,0,0,0,0,0,0,0,0,0


First, we take a smaller dataset, as some of the models work best with small datasets and we want the same one for all the models.

In [ ]:
data_small = data[0:13000]

We check that the proportions of the target variable remain the same.

In [ ]:
len(data[data.Delay == 1])

12005

In [ ]:
len(data[data.Delay == 0])

14899

In [ ]:
len(data_small[data_small.Delay == 1])

5856

In [ ]:
len(data_small[data_small.Delay == 0])

7144

Therefore, the prortion of delayed flights in the original dataset is 12005/26904 ~ 0.45 and for the smaller dataset is 5856/13000 ~ 0.45.

Then we split our data into training (which includes validation) and test. We divide the training and test sets into X_train and X_test, which contains the explanatory variables and y_train and y_test, containing the target variable. We will use stratified partitions, which means that we will keep the same proportions of the target variable in each partition.

In [ ]:
X = data_small.loc[:, data_small.columns != "Delay"]
y = data_small.loc[:,['Delay']]
X_train, X_test, y_train, y_test = train_test_split(X, y,  
                                                    test_size=0.3, stratify=data_small.loc[:,['Delay']], random_state=1)
X_train.shape, X_test.shape

((9100, 594), (3900, 594))

We build a results table, so we can compare the metrics for each model.

In [ ]:
results = pd.DataFrame(columns=['Accuracy','Precision','Recall (mean)','F1-score (mean)', 'Time(s)'])

# Logistic Regression

We also build a results table for each modeling method, apart from the general one.

In [ ]:
results_lr = pd.DataFrame(columns=['Accuracy','Precision','Recall (mean)','F1-score (mean)', 'Time(s)'])

We first check if all the variables are significant.

In [ ]:
model_glm = sm.GLM.from_formula("Delay~ Airline+Flight+AirportFrom+AirportTo+DayOfWeek+Length+Time", data = flights)
result_glm = model_glm.fit()
result_glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Delay   No. Observations:                26904
Model:                            GLM   Df Residuals:                    26317
Model Family:                Gaussian   Df Model:                          586
Link Function:               identity   Scale:                         0.21800
Method:                          IRLS   Log-Likelihood:                -17387.
Date:                Mon, 06 Jun 2022   Deviance:                       5737.0
Time:                        08:04:23   Pearson chi2:                 5.74e+03
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.2557      0.198      1.289      0.197      -0.133       0.645
Airline[T.AA]         -0.0683      0.028     -2.399      0.016      -0.124      -0.013
Airline[T.AS]         -0.0803      0.037     -2.149      0.032      -0.154      -0.007
Airline[T.B6]          0.0938      0.031      3.048      0.002       0.033       0.154
Airline[T.CO]          0.1359      0.032      4.182      0.000       0.072       0.200
Airline[T.DL]          0.0313      0.022      1.432      0.152      -0.012       0.074
Airline[T.EV]          0.0766      0.024      3.138      0.002       0.029       0.124
Airline[T.F9]         -0.0190      0.038     -0.499      0.617      -0.094       0.056
Airline[T.FL]         -0.0941      0.028     -3.336      0.001      -0.149      -0.039
Airline[T.HA]          0.0552      0.053      1.049      0.294      -0.048       0.158
Airline[T.MQ]         -0.0508      0.024     -2.124      0.034      -0.098      -0.004
Airline[T.OH]         -0.0730      0.028     -2.624      0.009      -0.128      -0.018
Airline[T.OO]          0.0326      0.025      1.322      0.186      -0.016       0.081
Airline[T.UA]         -0.1512      0.030     -5.008      0.000      -0.210      -0.092
Airline[T.US]         -0.0613      0.028     -2.165      0.030      -0.117      -0.006
Airline[T.WN]          0.3247      0.025     13.174      0.000       0.276       0.373
Airline[T.XE]         -0.0173      0.027     -0.635      0.525      -0.071       0.036
Airline[T.YV]         -0.1147      0.029     -3.983      0.000      -0.171      -0.058
AirportFrom[T.ABI]    -0.0023      0.232     -0.010      0.992      -0.456       0.452
AirportFrom[T.ABQ]    -0.0725      0.136     -0.533      0.594      -0.339       0.194
AirportFrom[T.ABR]    -0.2422      0.485     -0.499      0.618      -1.193       0.709
AirportFrom[T.ABY]    -0.3976      0.220     -1.807      0.071      -0.829       0.034
AirportFrom[T.ACT]     0.1359      0.232      0.587      0.557      -0.318       0.590
AirportFrom[T.ACV]     0.1187      0.186      0.639      0.523      -0.245       0.483
AirportFrom[T.ACY]     0.2212      0.355      0.623      0.534      -0.475       0.918
AirportFrom[T.ADQ]    -0.1025      0.258     -0.397      0.692      -0.609       0.404
AirportFrom[T.AEX]     0.0180      0.188      0.096      0.923      -0.350       0.386
AirportFrom[T.AGS]     0.0388      0.166      0.234      0.815      -0.287       0.365
AirportFrom[T.ALB]    -0.0395      0.146     -0.271      0.787      -0.325       0.246
AirportFrom[T.AMA]    -0.0020      0.173     -0.012      0.991      -0.342       0.338
AirportFrom[T.ANC]    -0.0442      0.159     -0.278      0.781      -0.356       0.267
AirportFrom[T.ASE]    -0.0847      0.162     -0.521      0.602      -0.403       0.234
AirportFrom[T.ATL]     0.0426      0.1

One of the Logistic Regression Assumptions is that only meaningful variables should be included. Therefore, since the p-value of the variable Flight is bigger that 0.05, we will remove it. 

In [ ]:
X_train_lr = X_train.drop(['Flight'],axis=1)
X_train_lr
X_test_lr = X_train.drop(['Flight'],axis=1)
X_test_lr

,Unnamed: 0,Length,Time,Speed,Unnamed: 0,DayOfWeek,Speed,Airline_9E,Airline_AA,Airline_AS,...,AirportTo_TXK,AirportTo_TYR,AirportTo_TYS,AirportTo_UTM,AirportTo_VLD,AirportTo_VPS,AirportTo_WRG,AirportTo_XNA,AirportTo_YAK,AirportTo_YUM
4490,4491,-1.247,1.440,-1.173,4491,5,-1.173,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4355,4356,1.445,-0.957,2.005,4356,3,2.005,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11524,11525,-1.191,-0.417,-0.547,11525,7,-0.547,0,0,0,...,0,0,0,0,0,0,0,0,0,0
620,621,1.194,-0.769,1.399,621,4,1.399,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4347,4348,0.865,-0.769,1.056,4348,2,1.056,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12847,12848,0.511,0.028,-0.014,12848,2,-0.014,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1723,1724,-1.417,0.407,-0.946,1724,3,-0.946,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9173,9174,0.475,0.319,-0.240,9174,2,-0.240,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1922,1923,0.055,-1.325,0.980,1923,1,0.980,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,Unnamed: 0,Length,Time,Speed,Unnamed: 0,DayOfWeek,Speed,Airline_9E,Airline_AA,Airline_AS,...,AirportTo_TXK,AirportTo_TYR,AirportTo_TYS,AirportTo_UTM,AirportTo_VLD,AirportTo_VPS,AirportTo_WRG,AirportTo_XNA,AirportTo_YAK,AirportTo_YUM
4490,4491,-1.247,1.440,-1.173,4491,5,-1.173,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4355,4356,1.445,-0.957,2.005,4356,3,2.005,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11524,11525,-1.191,-0.417,-0.547,11525,7,-0.547,0,0,0,...,0,0,0,0,0,0,0,0,0,0
620,621,1.194,-0.769,1.399,621,4,1.399,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4347,4348,0.865,-0.769,1.056,4348,2,1.056,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12847,12848,0.511,0.028,-0.014,12848,2,-0.014,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1723,1724,-1.417,0.407,-0.946,1724,3,-0.946,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9173,9174,0.475,0.319,-0.240,9174,2,-0.240,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1922,1923,0.055,-1.325,0.980,1923,1,0.980,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now we adjust a logistic classifier to our training data, and we perform 5-fold cross-validation to find the accuracy, precision, recall and f1-score.

In [ ]:
init_time = time()
# create the logistic classifier
logreg = LogisticRegression()
#train the model with the training set
logreg.fit(X_train_lr, y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(logreg, X_train_lr, y_train, cv=5)
scores_precision = cross_val_score(logreg, X_train_lr, y_train, cv=5,scoring='precision_macro')
scores_recall = cross_val_score(logreg, X_train_lr, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(logreg, X_train_lr, y_train, cv=5,scoring='f1_macro')
results.loc['Logistic Regression',:] = [np.mean(scores),np.mean(scores_precision),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_lr.loc['Logistic Regression',:] = [np.mean(scores),np.mean(scores_precision),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_lr

LogisticRegression()

0:00:04.083163


,Accuracy,Precision,Recall (mean),F1-score (mean),Time(s)
Logistic Regression,0.633,0.635,0.615,0.609,4.083


# SVM

We build a results table to compare the different SVMs we are going to try.

In [ ]:
results_svm = pd.DataFrame(columns=['Accuracy','Precision','Recall (mean)','F1-score (mean)', 'Time(s)'])

Fistly, we run our SVM with the default parameters.

In [ ]:
init_time = time()
svm = SVC()
svm.fit(X_train,y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(svm, X_train, y_train, cv=5)
scores_precision = cross_val_score(svm, X_train, y_train, cv=5,scoring='precision_macro')
scores_recall = cross_val_score(svm, X_train, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(svm, X_train, y_train, cv=5,scoring='f1_macro')
results_svm.loc['SVM-default',:] = [np.mean(scores),np.mean(scores_precision),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_svm

SVC()

0:00:56.198032


,Accuracy,Precision,Recall (mean),F1-score (mean),Time(s)
SVM-default,0.55,0.275,0.5,0.355,56.198


SVM has a function to deal with unbalanced data, we can weight the C hyperparameter based on the number of samples of each class, penalyzing this way the majoritary class.

In [ ]:
init_time = time()
svm = SVC(class_weight='balanced')
svm.fit(X_train,y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(svm, X_train, y_train, cv=5)
scores_precision = cross_val_score(svm, X_train, y_train, cv=5,scoring='precision_macro')
scores_recall = cross_val_score(svm, X_train, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(svm, X_train, y_train, cv=5,scoring='f1_macro')
results_svm.loc['SVM-balanced',:] = [np.mean(scores),np.mean(scores_precision),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_svm

SVC(class_weight='balanced')

0:01:01.397659


,Accuracy,Precision,Recall (mean),F1-score (mean),Time(s)
SVM-default,0.55,0.275,0.5,0.355,56.198
SVM-balanced,0.498,0.527,0.52,0.481,61.398


Therefore, we decide the best SVM model is the one with the default parameters, as we use the accuracy metric (also the other ones are mostly better and the time is practically the same). We add the chosen model to the general results table.

In [ ]:
results.loc['SVM',:] = results_svm.loc['SVM-default',:]

# Random Forest


In [ ]:
results_rf = pd.DataFrame(columns=['Accuracy','Precision','Recall (mean)','F1-score (mean)', 'Time(s)'])

We create the model and we fit it with our training data.

In [ ]:
init_time = time()
randomforest = RandomForestClassifier(oob_score=True)
randomforest.fit(X_train, y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(randomforest, X_train, y_train, cv=5)
scores_precision = cross_val_score(randomforest, X_train, y_train, cv=5,scoring='precision_macro')
scores_recall = cross_val_score(randomforest, X_train, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(randomforest, X_train, y_train, cv=5,scoring='f1_macro')
results_rf.loc['Random Forest',:] = [np.mean(scores),np.mean(scores_precision),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_rf
print('OOB accuracy=', randomforest.oob_score_)

RandomForestClassifier(oob_score=True)

0:00:06.901300


,Accuracy,Precision,Recall (mean),F1-score (mean),Time(s)
Random Forest,0.627,0.62,0.618,0.615,6.901


OOB accuracy= 0.6247252747252747


Now we try tuning different hyperparameters:

In [ ]:
init_time = time()

randomforest = RandomForestClassifier(oob_score=True)

ntrees = [200,None]
max_depth = [100,None]
min_samples_split = [4,6]
min_samples_leaf = [4,6]
balance = [None, 'balanced', 'balanced_subsample']

crossval = GridSearchCV(estimator=randomforest,
                   scoring=['accuracy'],
                   param_grid={
                       'n_estimators': ntrees,
                       'max_depth':max_depth,
                       'min_samples_split':min_samples_split,
                       'min_samples_leaf':min_samples_leaf, 
                       'class_weight':balance
                   },
                   cv=5,
                   refit='accuracy')

randomforest_param = crossval.fit(X_train, y_train)
print(timedelta(seconds=(time() - init_time)))

0:11:31.259391


And we find the best ones (in terms of accuracy):

In [ ]:
best_params=randomforest_param.best_params_
best_params

{'class_weight': None,
 'max_depth': None,
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 200}

Then we fit another Random Forest model with the best parameters:

In [ ]:
init_time = time()
randomforest = RandomForestClassifier(oob_score=True,**best_params)
randomforest.fit(X_train, y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(randomforest, X_train, y_train, cv=5)
scores_precision = cross_val_score(randomforest, X_train, y_train, cv=5,scoring='precision_macro')
scores_recall = cross_val_score(randomforest, X_train, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(randomforest, X_train, y_train, cv=5,scoring='f1_macro')
results_rf.loc['Random Forest-best',:] = [np.mean(scores),np.mean(scores_precision),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_rf
print('OOB accuracy=', randomforest.oob_score_)

RandomForestClassifier(min_samples_leaf=4, min_samples_split=4,
                       n_estimators=200, oob_score=True)

0:00:12.273959


,Accuracy,Precision,Recall (mean),F1-score (mean),Time(s)
Random Forest,0.627,0.62,0.618,0.615,6.901
Random Forest-best,0.641,0.65,0.619,0.611,12.274


OOB accuracy= 0.6463736263736264


We see that the best one is the second one, the one with the tuned hyperparameters, so that is the Random Forest model we choose. We add it to the general results table.

In [ ]:
results.loc['Random Forest',:] = results_rf.loc['Random Forest-best',:]

# Naive-Bayes classifier


In [ ]:
results_nb = pd.DataFrame(columns=['Accuracy','Precision','Recall (mean)','F1-score (mean)', 'Time(s)'])

We start fitting a Gaussian Naive-Bayes classifier with the default parameters.

In [ ]:
init_time = time()
#creation of the classifier
naive_bayes = GaussianNB()
#training of the model
naive_bayes.fit(X_train, y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(naive_bayes, X_train, y_train, cv=5)
scores_precision = cross_val_score(naive_bayes, X_train, y_train, cv=5,scoring='precision_macro')
scores_recall = cross_val_score(naive_bayes, X_train, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(naive_bayes, X_train, y_train, cv=5,scoring='f1_macro')
results_nb.loc['Naive-Bayes',:] = [np.mean(scores),np.mean(scores_precision),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_nb

GaussianNB()

0:00:00.112957


,Accuracy,Precision,Recall (mean),F1-score (mean),Time(s)
Naive-Bayes,0.59,0.596,0.596,0.589,0.113


Then we try hyperparameter tuning to improve accuracy and we find the best parameters:

In [ ]:
np.logspace(0,-9, num=10)

array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06, 1.e-07,
       1.e-08, 1.e-09])

In [ ]:
naive_bayes = GaussianNB()
params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

gnb_cv = GridSearchCV(estimator=naive_bayes, 
                     param_grid=params_NB, 
                     cv=5,
                     scoring='accuracy')

gnb_cv.fit(X_train, y_train);

In [ ]:
best_params = gnb_cv.best_params_
best_params

{'var_smoothing': 6.579332246575682e-09}

Now we refit with the best parameter found:

In [ ]:
init_time = time()
best_gnbmodel = GaussianNB(**best_params)
best_gnbmodel.fit(X_train, y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(best_gnbmodel, X_train, y_train, cv=5)
scores_precision = cross_val_score(best_gnbmodel, X_train, y_train, cv=5,scoring='precision_macro')
scores_recall = cross_val_score(best_gnbmodel, X_train, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(best_gnbmodel, X_train, y_train, cv=5,scoring='f1_macro')
results_nb.loc['Naive-Bayes-best',:] = [np.mean(scores),np.mean(scores_precision),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_nb

GaussianNB(var_smoothing=6.579332246575682e-09)

0:00:00.111484


,Accuracy,Precision,Recall (mean),F1-score (mean),Time(s)
Naive-Bayes,0.59,0.596,0.596,0.589,0.113
Naive-Bayes-best,0.621,0.619,0.619,0.619,0.111


Additionally, we will perform a Power Transform to the data and fit again a model with the best hyperparameter.

In [ ]:
naive_bayes = GaussianNB()
params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

gnb_cv = GridSearchCV(estimator=naive_bayes, 
                     param_grid=params_NB, 
                     cv=5,
                     scoring='accuracy')

Data_transformed = PowerTransformer().fit_transform(X_train)

gnb_cv.fit(Data_transformed, y_train);

In [ ]:
best_params = gnb_cv.best_params_
best_params

{'var_smoothing': 0.15199110829529336}

We refit with the parameters found and the data transformed:

In [ ]:
init_time = time()
best_gnbmodel = GaussianNB(**best_params)
best_gnbmodel.fit(Data_transformed, y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(best_gnbmodel, Data_transformed, y_train, cv=5)
scores_precision = cross_val_score(best_gnbmodel, Data_transformed, y_train, cv=5,scoring='precision_macro')
scores_recall = cross_val_score(best_gnbmodel, Data_transformed, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(best_gnbmodel, Data_transformed, y_train, cv=5,scoring='f1_macro')
results_nb.loc['Naive-Bayes-best-transf',:] = [np.mean(scores),np.mean(scores_precision),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_nb

GaussianNB(var_smoothing=0.15199110829529336)

0:00:00.104827


,Accuracy,Precision,Recall (mean),F1-score (mean),Time(s)
Naive-Bayes,0.59,0.596,0.596,0.589,0.113
Naive-Bayes-best,0.621,0.619,0.619,0.619,0.111
Naive-Bayes-best-transf,0.497,0.571,0.534,0.443,0.105


The best Naive-Bayes classifier is the one with the tuned hyperparameter. 

In [ ]:
results.loc['Naive-Bayes',:]=results_nb.loc['Naive-Bayes-best',:]

Finally, we print the general results table:

In [ ]:
results

,Accuracy,Precision,Recall (mean),F1-score (mean),Time(s)
Logistic Regression,0.633,0.635,0.615,0.609,4.083
SVM,0.55,0.275,0.5,0.355,56.198
Random Forest,0.641,0.65,0.619,0.611,12.274
Naive-Bayes,0.621,0.619,0.619,0.619,0.111


# Predictions with the chosen model

We refit our chosen model with the full training data and the chosen hyperparameters.

In [ ]:
FinalModel = RandomForestClassifier(oob_score=True,class_weight=None,max_depth=None,
                                    min_samples_leaf=4,min_samples_split=4,n_estimators=200)
FinalModel.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=4, min_samples_split=4,
                       n_estimators=200, oob_score=True)

Next, we predict the test set results and calculate the accuracy with our final model:

In [ ]:
# prediction with the test dataset
y_pred = FinalModel.predict(X_test)

We now print the confusion matrix

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[1808  335]
 [1102  655]]


Therefore, we have 1808+655 correct predictions and 335+1102 incorrect ones. We can also see that it is easier to predict a flight being delayed than otherwise and that our model is quite likely to wrongly predict the flight not being delayed. 


Lastly, we compute the accuracy, precision, recall, F-measure and support:

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.84      0.72      2143
           1       0.66      0.37      0.48      1757

    accuracy                           0.63      3900
   macro avg       0.64      0.61      0.60      3900
weighted avg       0.64      0.63      0.61      3900

